In [1]:
!pip install pymongo[srv]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 21.4 MB/s eta 0:00:00


In [ ]:
# !pip install "pymongo[srv]"==3.6

In [2]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [3]:
import pymongo
from pymongo import MongoClient
import json

In [4]:
# Connect to the MongoDB Atlas cluster
# Replace the <connection_string> with that of the cluster
client = MongoClient("<connection_string>")

In [5]:
# Access the sample_mflix database
db = client.sample_mflix

In [6]:
# Function to extract schema for a given collection
def get_collection_schema(collection_name):
    collection = db[collection_name]
    sample_document = collection.find_one()
    schema = {}
    for field, value in sample_document.items():
        field_type = type(value).__name__
        if isinstance(value, list):
            if value:
                if isinstance(value[0], dict):
                    inner_fields = {}
                    for inner_doc in value:
                        inner_fields.update(get_collection_schema(collection_name))
                    schema[field] = {"type": "array", "description": f"List of objects with fields: {inner_fields}"}
                else:
                    inner_type = type(value[0]).__name__
                    schema[field] = {"type": "array", "description": f"List of {inner_type}s"}
            else:
                schema[field] = {"type": "array"}
        else:
            schema[field] = {"type": field_type}
    return schema

In [7]:
# Function to generate the JSON schema for all collections
def generate_schema():
    collections_schema = {}
    for collection_name in db.list_collection_names():
        collections_schema[collection_name] = {"name": collection_name, "description": f"Contains detailed information about {collection_name}."}
        collections_schema[collection_name]["fields"] = get_collection_schema(collection_name)
    return collections_schema

In [8]:
# Generate the schema
schema = generate_schema()

In [9]:
schema

{'sessions': {'name': 'sessions',
  'description': 'Contains detailed information about sessions.',
  'fields': {'_id': {'type': 'ObjectId'},
   'user_id': {'type': 'str'},
   'jwt': {'type': 'str'}}},
 'movies': {'name': 'movies',
  'description': 'Contains detailed information about movies.',
  'fields': {'_id': {'type': 'ObjectId'},
   'plot': {'type': 'str'},
   'genres': {'type': 'array', 'description': 'List of strs'},
   'runtime': {'type': 'int'},
   'cast': {'type': 'array', 'description': 'List of strs'},
   'poster': {'type': 'str'},
   'title': {'type': 'str'},
   'fullplot': {'type': 'str'},
   'languages': {'type': 'array', 'description': 'List of strs'},
   'released': {'type': 'datetime'},
   'directors': {'type': 'array', 'description': 'List of strs'},
   'rated': {'type': 'str'},
   'awards': {'type': 'dict'},
   'lastupdated': {'type': 'str'},
   'year': {'type': 'int'},
   'imdb': {'type': 'dict'},
   'countries': {'type': 'array', 'description': 'List of strs'},
 

In [11]:
# Write the schema to a JSON file
with open("movie.json", "w") as f:
    json.dump({"collections": schema}, f, indent=4)

print("Schema generation complete. Schema saved as 'movie.json'.")

Schema generation complete. Schema saved as 'movie.json'.
